In [17]:
import geojson
import leafmap.leafmap as leafmap
import xarray as xr
import rasterio
import geopandas as gpd

## Vector data - critical assets

In [11]:
data_dir = "C:/Users/frans/Documents/NCL/R2D2/tests/data/"
## Polygon - NE boundary
with open(data_dir+"north_east_4326.geojson") as json_file:
   north_east_lim = geojson.load(json_file)

## Centroids - Hospitals
with open(data_dir+"osm/hospitals_centroids.geojson") as json_file:
   hospitals = geojson.load(json_file)

## Centroids - Schools
with open(data_dir+"osm/schools_centroids.geojson") as json_file:
   schools = geojson.load(json_file)

## Centroids - Halls
with open(data_dir+"osm/halls_centroids.geojson") as json_file:
   halls = geojson.load(json_file)

## Centroids - Sport centres
with open(data_dir+"osm/sports_centre_centroids.geojson") as json_file:
   sport_centres = geojson.load(json_file)

## Points - Care homes
with open(data_dir+"osm/care_homes.geojson") as json_file:
   care_homes = geojson.load(json_file)

In [ ]:
m = leafmap.Map(center=(55, -1.9), zoom=7, width="800px")
m.add_basemap('CartoDB.DarkMatter')
#m.add_basemap("Stamen.TonerLite")
m.add_geojson(north_east_lim, layer_name="North East")
m

## Raster data - weather

In [18]:
storm_darcy = "snow_products/stormDarcy/storm_darcy_"
tbfte = "snow_products/TBFTE/bfte_"

snow_dir = storm_darcy


In [19]:
# Load the snow depth raster
with rasterio.open(snow_dir +"era5_snow_cover.tif") as src:
    snow_cover= src.read(1)
    transform = src.transform
    crs = src.crs

print(crs)
print(snow_cover.shape)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "rasterio\_base.pyx", line 308, in rasterio._base.DatasetBase.__init__
  File "rasterio\_base.pyx", line 219, in rasterio._base.open_dataset
  File "rasterio\_err.pyx", line 221, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_OpenFailedError: snow_products/stormDarcy/storm_darcy_era5_snow_cover.tif: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\frans\miniconda3\envs\geo\Lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\frans\AppData\Local\Temp\ipykernel_7712\4097205003.py", line 2, in <module>
    with rasterio.open(snow_dir +"era5_snow_cover.tif") as src:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\frans\miniconda3\envs\geo\Lib\site-packages\rasterio\env.py", line 451, in wrapper
    return f(*args, **kwds)
      

In [8]:
#raster ='bfte_ndsi_snow_cover.tif'
darcy_snow_cover = "snow_products/stormDarcy"
m.add_raster(raster, layer_name='ndsi',)

In [ ]:
#labels = ['Hospitals', 'Schools', 'Residential']
#colors = ['#ff0000', '#9900cc', '#006600']
labels = ['Hospitals', 'Schools']
colors = ['#ff0000', '#9900cc']
hover_style = {"fillOpacity": 0.7}
m.add_geojson(hospitals, layer_name=labels[0], style={'color':colors[0]}, hover_style=hover_style)
m.add_geojson(schools, layer_name=labels[1], style={'color':colors[1],},hover_style=hover_style)
#m.add_geojson(residential, layer_name=labels[2], style={'color':colors[2],},hover_style=hover_style)
m.add_legend(labels = labels, colors=colors)

In [ ]:
#raster = xr.open_rasterio("2018_02_28_20.tif")
raster ='2018_02_28_20.tif'
m.add_raster(raster, layer_name='snow_depth',)

## Analysis

In [ ]:
# Load the snow depth raster
with rasterio.open('2018_02_28_20.tif') as src:
    snow_depth = src.read(1)
    transform = src.transform
    crs = src.crs

print(crs)
print(snow_depth.shape)

# Load the school and hospital polygons
schools_gpd = gpd.read_file('C:/Users/frans/Documents/NCL/R2D2/data/osm/school.geojson')
hospitals_gpd = gpd.read_file('C:/Users/frans/Documents/NCL/R2D2/data/osm/hospital.geojson')
print(schools_gpd.crs)

In [ ]:
# Reproject the school and hospital polygons to the same CRS as the snow depth raster
#schools_gpd = schools_gpd.to_crs(crs)
#hospitals_gpd = hospitals_gpd.to_crs(crs)

In [ ]:
schools_gpd.head(3)

In [ ]:
# Sample the snow depth raster at the centroid of each school and hospital polygon
threshold = 0.06
for idx, row in hospitals_gpd.iterrows():
    point = row.geometry.centroid
    col, r = src.index(point.x, point.y)
    if col < 0 or col > src.width-1 or r < 0 or r > src.height-1:
        print("Point is outside the bounds of the raster.")
    else:
        value = snow_depth[r, col]
        #print(value)
        if value > threshold:
            #print("TRUE")
            print(f"Hospital {idx} has snow depth {value:.2f}")
            #print(f"Hospital {row['FID_1']} has snow depth {value:.2f}")

In [ ]:
for idx, row in schools_gpd.iterrows():
    point = row.geometry.centroid
    col, r = src.index(point.x, point.y)
    if col < 0 or col > src.width-1 or r < 0 or r > src.height-1:
        print("Point is outside the bounds of the raster.")
    else:
        value = snow_depth[r, col]
        #print(value)
        if value > threshold:
            #print("TRUE")
            print(f"School {idx} has snow depth {value:.2f}")
            #print(f"School {row['FID_1']} has snow depth {value:.2f}")